# Importing libraries

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np
import time

# initiating ChromeDriver -- To control Chrome

In [78]:
# making browser headless
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')

# initiating ChromeDriver -- To control Chrome
driver = webdriver.Chrome(executable_path='F:/__Mix Soft 1/chromedriver.exe', options=options)

C:\Python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  import sys


# Requesting the page

In [79]:
# requesting the page
url = 'https://twitter.com/ImranKhanPTI/status/1614535943737823232?cxt=HHwWgMC-ud_4_ecsAAAA'
driver.get(url)

# defualt wait the elements to load 
driver.implicitly_wait(20)

# 

In [56]:
def show_more_tweets():
    show_more = driver.find_element(By.XPATH, '//span[.="Show more replies"]')
    location = show_more.location_once_scrolled_into_view
   
    location['y'] -= 200
    driver.execute_script(f'window.scrollBy(0, {location["y"]});')
    show_more.click()
    time.sleep(5)



# 

In [ ]:

def scrape(tweets):
    tweet_boxes = driver.find_elements(By.CSS_SELECTOR, "article[data-testid='tweet']")
  
    for index, tweet_box in enumerate(tweet_boxes):
        try:
            user_details = tweet_box.find_element(By.CSS_SELECTOR, "div[data-testid='User-Names'")
        except:
            print(tweet_box.text)
            break
        user_name = user_details.find_element(By.XPATH, './div')
        user_handle = user_name.find_element(By.XPATH , 'following-sibling::div/div/div')
        try:
            date = user_handle.find_element(By.XPATH, './following-sibling::div/following-sibling::div')
        except:
            continue
        
        try:
            tweet = tweet_box.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
    
        except:
            continue
        tweet = tweet_box.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
        
        temp = tweet.find_element(By.XPATH, '..')
        temp = temp.find_elements(By.XPATH, './following-sibling::div')
        if (len(temp) == 0):
            continue
        tweet_data = temp[-1].find_element(By.XPATH, './div')
        replies = tweet_data.find_element(By.XPATH, './div')
        retweets = replies.find_element(By.XPATH, './following-sibling::div')
        likes = replies.find_element(By.XPATH, 'following-sibling::div/following-sibling::div')
        reach = likes.find_element(By.XPATH, 'following-sibling::div')

          #parsing
        tweets['Date'].append(date.text)
        tweets['User Name'].append(user_name.text)
        tweets['User Handle'].append(user_handle.text)
        if(tweet.text == ''):
            tweets['Tweet'].append(np.nan)
        else:
            tweets['Tweet'].append(tweet.text)
        if(replies.text !='' and retweets.text !='' and likes.text !='' and reach.text !=''):
            replies = replies.text.replace(',','')
            retweets = retweets.text.replace(',','')
            likes = likes.text.replace(',','')
            reach = reach.text.replace(',','')

            if ('K' in replies):
                replies = int(replies.replace('K', '')) * 1000
                tweets['Replies'].append(replies)
            else:
                tweets['Replies'].append(int(replies))
            if ('K' in retweets):
                retweets = int(retweets.replace('K', '')) * 1000
                tweets['Retweets'].append(retweets)
            else:
                tweets['Retweets'].append(int(retweets))

            if ('K' in likes):
                likes = int(likes.replace('K', '')) * 1000
                tweets['Likes'].append(likes)
            else:
                tweets['Likes'].append(int(likes))
            if ('K' in reach):
                reach = int(reach.replace('K', '')) * 1000
                tweets['Reach'].append(reach)
            else:
                tweets['Reach'].append(int(reach))

        else:
            tweets['Replies'].append(replies.text)
            tweets['Retweets'].append(retweets.text)
            tweets['Likes'].append(likes.text)
            tweets['Reach'].append(reach.text)
            
      


# Main 

In [75]:
tweets = {"Date":[], "User Name":[], "User Handle":[], "Tweet":[], "Replies":[], "Retweets":[], "Likes":[], "Reach":[]}
for i in range(10):
    print('Scraping...')
    scrape(tweets)
    print('Getting More...')
    show_more_tweets()
    
scrape(tweets)



Scraping...
Getting More...


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x01066643]
	(No symbol) [0x00FFBE21]
	(No symbol) [0x00EFDA9D]
	(No symbol) [0x00EDEF6A]
	(No symbol) [0x00F53AAB]
	(No symbol) [0x00F661B6]
	(No symbol) [0x00F4FB76]
	(No symbol) [0x00F249C1]
	(No symbol) [0x00F25E5D]
	GetHandleVerifier [0x012DA142+2497106]
	GetHandleVerifier [0x013085D3+2686691]
	GetHandleVerifier [0x0130BB9C+2700460]
	GetHandleVerifier [0x01113B10+635936]
	(No symbol) [0x01004A1F]
	(No symbol) [0x0100A418]
	(No symbol) [0x0100A505]
	(No symbol) [0x0101508B]
	BaseThreadInitThunk [0x75BAF989+25]
	RtlGetAppContainerNamedObjectPath [0x77BB5DE4+228]
	RtlGetAppContainerNamedObjectPath [0x77BB5DB4+180]


In [ ]:
# output
df = pd.DataFrame(tweets)
df

In [58]:
uni = df.drop_duplicates()
uni.dropna()

,Date,User Name,User Handle,Tweet,Replies,Retweets,Likes,Reach
0,Jan 15,Ali.,@aliakberhabib,If only your party members would have come out...,4,1,65,7423
1,Jan 15,Saleem Rehman Ex-MNA,@mna_saleem,#KarachiKaptaanKa,2,3,32,3309
2,Jan 15,Nayyar Sohail,@nayyarsohail,"A big step for democracy. Vote, if used proper...",,,6,"2,066"
3,Jan 15,Abid Jani Afridi,@AJ_Afridi_UK,Everyone should come out for voting,,,17,"2,182"
4,Jan 15,Zulqarnain Ranjha,@SahabRanjha786,جس پیمانے پر کراچی الیکشن میں دھاندلی ہو رہی ی...,4,,7,"2,097"
14,Jan 15,MG Mughal,@Balance2Extreme,Good luckMr PM \n@ImranKhanPTI\n \nيا ذوالجلا...,1,,5,"1,900"
15,Jan 15,Stranger,@StrangerExpose,آؤ مل کر بنائیں پاکستان اور کرپٹ مافیا سے پاک ...,,,4,768
16,Jan 15,Sumaira Imtiaz,@chaynisar,,3,7,27,2569
17,7h,Imran Khan,@ImranKhanPTI,Strongly condemn burning of Holy Quran yesterd...,770,16200.0,41700.0,608200.0
18,Jan 21,Imran Khan,@ImranKhanPTI,This is rule of law where no one is above it; ...,1398,12300.0,36000.0,958000.0


In [67]:
show_more_tweets()